# Remote data

In this lesson we explore tools and components to interact with 
remote geospatial web-services to fetch information and geospatial data.

## OWSLib - Python client for OGC OWS

[OWSlib](https://geopython.github.io/OWSLib) is 
a client implemented in Python for Web Services standardized by the [Open Geospatial Consortium](https://www.opengeospatial.org/)
hence often nicknamed "OGC OWS".
 
`OWSLib` enables you to connect to these services as 
a client, mainly to fetch and query data from them. 
Currently WMS, WFS, WCS, CSW, WPS, SOS, WMC and the more recent OGC APIs like 
*OGC API - Features* (formerly called "WFS version 3") are supported.
The list of supported services is growing. 

Documentation: https://geopython.github.io/OWSLib/


## Interact with a WMS

OGC Web Map Service (WMS) can be used to download map images rendered by the remote server.

### GetCapabilities
This is the metadata of the service endpoint (thanks to our friends at Terrestris).

In [ ]:
from owslib.wms import WebMapService

wms_url = 'https://ows.terrestris.de/osm/service'

wms = WebMapService(wms_url, version="1.3.0")

print(f'WMS title: {wms.identification.title}')

print(f'WMS abstract: {wms.identification.abstract}')

print(f'Provider name: {wms.provider.name}')

print(f'Provider address: {wms.provider.contact.address}')

Check the [Capabilities response](https://ows.terrestris.de/osm/service?service=WMS&request=GetCapabilities&version=1.3.0) directly from the server

Available WMS layers:

In [ ]:
list(wms.contents)

In [ ]:
print(wms.contents['SRTM30-Colored'].boundingBox)

In [ ]:
print(wms.contents['OSM-WMS'].boundingBoxWGS84)

Download and save the data (image)

In [ ]:
img = wms.getmap(
    layers=['SRTM30-Colored'],
    size=[600, 400],
    srs="EPSG:4326",
    bbox=[1.0, 50.0, 10.0, 54.0],
    format="image/jpeg")

save_fp = 'test/10-wms.png'
with open(save_fp, 'wb') as out:
	out.write(img.read())

Display image in Notebook or view in browser via [this link](test/10-wms.png).

In [ ]:
from IPython.display import Image
Image(filename=save_fp)

## Interact with a WFS

OGC Web Feature Service (WFS) can be used to download vector data (called *Features*) from a remote server.
In [Lesson 04 - Vector Data](04-vector-data.ipynb) we have learned that vector data
can come in various formats. With WFS the default format is GML, both "flat", record-like Simple Features,
but also as more complex GML application schema data. Most WFS implementations though, notably GeoServer, will
also allow you to download data in additional vector formats such as GeoJSON and even as ESRI Shapefiles.

When talking about "WFS", the OGC WFS version 1 and 2 is meant. Recently OGC has focused efforts on modernizing the  OWS specifications, which are a total rewrite (using REST, OpenAPI and GeoJSON). As a result, WFS has been rebranded as "OGC API - Features".
This will be treated further below.

The terminology for WFS is different as for WMS. Where WMS has **Layers**, a WFS has equivalent **FeatureTypes** (could even be from the same data, e.g. a database table) basically a collection of **Features**.
 
### GetCapabilities
This is the metadata of the WFS endpoint.


In [ ]:
from owslib.wfs import WebFeatureService

wfs_url = 'https://ahocevar.com/geoserver/wfs'

wfs = WebFeatureService(wfs_url, version='2.0.0')

In [ ]:
print(f'WFS title: {wfs.identification.title}')

print(f'WFS abstract: {wfs.identification.abstract}')

print(f'Provider name: {wfs.provider.name}')

print(f'Provider address: {wfs.provider.contact.address}')

Check the [Capabilities response](https://ahocevar.com/geoserver/wfs?service=WFS&request=GetCapabilities&version=2.0.0) directly from the server

Available operations (services):

In [ ]:
[operation.name for operation in wfs.operations]

Available WFS FeatureTypes:

In [ ]:
list(wfs.contents)

As a Feature is basically a (sometime one or more) geometry with attribute data, we would be interested
in the the *schema*. For this a WFS provides a `DescribeFeatureType` service. 

In [ ]:
wfs.contents['usa:states'].boundingBox

With the `GetFeature` service you can fetch, basically query, for vector data.
This is the key operation of WFS. It is very powerful as well in terms
of its query-options (parameters). You can basically think of a WFS as a web-accessible spatial database.

As a minimum the `typename` parameter is required. Other parameters include:

* `srsname`: fetch (reproject) data according to SRS (usually the EPSG code)
* `bbox`: get the data contained in specified bounding box
* `filter`: this allows you to filter out data, basically like SQL
* `outputFormat`: download the data in various Vector data formats (default GML)

Example. Download all "populated places" within the bounding box of Romania as GML using the `GetFeature` operation.
Tip: see https://gist.github.com/graydon/11198540 for country bounding boxes.

`OWSLib` will switch the axis order from EN to NE automatically if designated by EPSG-Registry.


In [ ]:
response = wfs.getfeature(typename='ne:ne_10m_populated_places', bbox=(20.2201, 43.6884, 29.6265, 48.2208))

Now download in a more handy format like GeoJSON (mind: not all WFS-es support alternative output formats!)

In [ ]:
response = wfs.getfeature(typename='ne:ne_10m_populated_places', bbox=(20.2201, 43.6884, 29.6265, 48.2208), outputFormat='json')
with open('test/10-populated-places-ro.json', 'w', encoding='UTF-8') as out:
	out.write(str(response.read(), 'UTF-8'))

Display data in Notebook or view in browser via [this link](test/10-populated-places-ro.json).

Download GML data using `typename` and `filter` (get only a feature with `NAME=Bucharest`). 
OWSLib currently only supports filter building for WFS 1.1 (FE.1.1).

In [ ]:
from owslib.fes import PropertyIsLike
from owslib.etree import etree
wfs11 = WebFeatureService(url='https://ahocevar.com/geoserver/wfs?', version='1.1.0')

filter_ = PropertyIsLike(propertyname='NAME', literal='Bucharest', wildCard='*')
filterxml = etree.tostring(filter_.toXML()).decode('utf-8')
response = wfs11.getfeature(typename='ne:ne_10m_populated_places', filter=filterxml)

Showing the GML data:

In [ ]:
str(response.read(), 'UTF-8')

Alternatively as GeoJSON. For you to decide which format is handier...

In [ ]:
response = wfs11.getfeature(typename='ne:ne_10m_populated_places', filter=filterxml, outputFormat='json')
json_str = str(response.read(), 'UTF-8')
import json
parsed = json.loads(json_str)
print(json.dumps(parsed, indent=2, sort_keys=True))

## Fetch from OGC API Features
Although this may sound very advanced, this is actually one of 
the simpler OGC APIs. See also the [OWSLib Manual for OAFeat](https://geopython.github.io/OWSLib/usage.html#ogc-api).


In [ ]:
from owslib.ogcapi.features import Features
oa_feat = Features('https://demo.pygeoapi.io/master')
oa_feat.links

In [ ]:
# Open API Specification v3 - API document
api = oa_feat.api() # OpenAPI definition
api

Or, if available, from your locally running `pygeoapi` server, which has the address `geopython-workshop-pygeoapi` within the Docker network.

In [ ]:
from owslib.ogcapi.features import Features
oa_feat = Features('http://geopython-workshop-pygeoapi')

In [ ]:
# Conformance stuff
print(f'This OGC API Features endpoint conforms to {oa_feat.conformance()}')

In [ ]:
# Get collections (datasets) in endpoint
collections = oa_feat.collections()
print(f'This OGC API Features endpoint has {len(collections)} datasets')

In [ ]:
# Get items (paged) in Lakes collection
lakes = oa_feat.collection('lakes')
lakes_query = oa_feat.collection_items('lakes')
lakes_query['features'][0]

## Searching for the data in CSW server - OPTIONAL
This last section is quite advanced, you may want to skip if
time pressures.

We will use a metadata service (CSW) to find our target 
data service, a regular WFS (v2). All via OWSLib.

### Step 1 - Find WFS endpoints by querying a CSW endpoint

First step: let's find some WFS service in our CSW server:
NOTE: You can skip this step and use the URL directly


In [ ]:
from owslib.fes import PropertyIsLike, BBox, And, PropertyIsEqualTo
from owslib.csw import CatalogueServiceWeb

thecsw = CatalogueServiceWeb('http://geoportal.gov.cz/php/micka/csw/index.php')

In [ ]:
# wfs_query = PropertyIsLike('csw:AnyText', 'WFS')
geology_query = PropertyIsLike('csw:AnyText', 'Geology')
service_query = PropertyIsLike('apiso:type', 'service')
geology_and_wfs = And([geology_query, service_query])
thecsw.getrecords2([geology_and_wfs], esn='full')

print(thecsw.results)

In [ ]:
for recid in thecsw.records:
    record = thecsw.records[recid]
    print(recid, record.title)

### Step 2 - Get the WFS endpoint and interact with it
Let's have a look at WFS data in czech Geology survey.

In [ ]:
geology = thecsw.records['575a4ef6-2f74-43ed-9352-6f400a010852']
print(f'Abstract: {geology.abstract}')

print(f'Identifier: {geology.identifiers[1]["identifier"]}')

In [ ]:
from owslib.wfs import WebFeatureService
url = 'http://inspire.geology.cz/geoserver/wms?service=WMS&version=1.3.0&request=Getcapabilities'
geology_wfs = WebFeatureService(geology.identifiers[1]['identifier'])

Service metadata (Capabilities):

In [ ]:
capabilities = geology_wfs.getcapabilities()
print(f'URL: {capabilities.geturl()}')
print(f'Name: {geology_wfs.provider.name}')
print(f'Title: {geology_wfs.identification.title}')
print(f'Keywords: {geology_wfs.identification.keywords[0]}')
print(f'Fees: {geology_wfs.identification.fees}')
print(f'Abstract: {geology_wfs.identification.abstract}')

Print list of available layers 

In [ ]:
for i in geology_wfs.contents:
    print(f'\n#### {i} ####')
    print(geology_wfs.contents[i].abstract)

Download data from selected layer and write to a file.

In [ ]:
identifier = 'gsmlp:CZE_CGS_500k_Fault'
features = geology_wfs.getfeature([identifier])

with open('test/10-geology-faults.gml', 'w', encoding='UTF-8') as out:
    out.write(str(features.read(), 'UTF-8'))

View the GML-data in browser via [this link](test/10-geology-faults.gml).

---
[<- Publishing](09-publishing.ipynb) | [Emerging technology and trends ->](11-emerging-technology-trends.ipynb)